`1. Initializing Elasticsearch and LLM client connection`

In [12]:
import os
from elasticsearch import Elasticsearch, helpers
from langchain_groq import ChatGroq

es_client = Elasticsearch(
    hosts="https://512baa73334c4127ade77bc2dfa2ef02.eastus2.azure.elastic-cloud.com:443",
    basic_auth=("elastic", "XRzYOABuG17Yv32UKAFySw41")
)

es_client.ping()

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    # api_key="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",
    api_key="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"
)

`2. Parsing the document and creating documents`

`2.1 Extracting Table of Content from the Doc`

In [13]:
from langchain.prompts import PromptTemplate
from PyPDF2 import PdfReader

from typing import List, Optional, Dict
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Index(BaseModel):
    Laws: Optional[List[str]] = Field(description="List of laws present in the legal document")
    Circulars: Optional[List[str]] = Field(description="List of circulars present in the legal document")
    Decrees: Optional[List[str]] = Field(description="List of decrees present in the legal document")
    ExecutiveCouncilResolutions: Optional[List[str]] = Field(description="List of executive council resolutions present in the legal document")


TOC_PROMPT = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an AI assistant specialized in identifying table of contents from the legal document paragraph.

    Guidelines:
    1. Thoroughly analyze the given document paragraph and generate the table of content.
    2. Table of content specifically target
        - Laws
        - Circulars
        - Decrees
        - ExecutiveCouncilResolutions
    3. Provide JSON response, where each dictionary key represents a primary topic (e.g., "Executive Council Resolutions," "Laws") and 
       each corresponding value contains a list of associated subtopics.
    4. Provide only the dictionary as the output, without any additional text, explanations, or commentary.
    5. If any primary topic is empty then exclude from the response and do not keep empty list.
    6. Do not change the format of the primary topic and associated subtopics, i.e., Punctuation, Comma, Full stop, quotation mark

    Sample response format:
    {{
      "Laws": ["Law No. (2) of 2022 Concerning the Reorganisation of the National Rehabilitation Centre - Abu Dhabi"],
      "ExecutiveCouncilResolutions": ["Chairman of the Executive Council Resolution No. (3) of 2022 Concerning the Appointment of the Director-General for Tourism"],
      "Circulars": ["Circular No. (1) of 2022 Concerning the Policy of Providing an Easy and Effortless Customer Experience in the Emirate of Abu Dhabi"],
      "Decrees": ["Amiri Decree No. (13) of 2023 Concerning Sending a Judge to Retirement"]
    }}

    Legal document paragraph: {paragraph}
    
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["paragraph"],
)

structured_llm = llm.with_structured_output(Index, method="json_mode")

toc_extractor = TOC_PROMPT | structured_llm

def table_of_content(reader):
    for page_no in range(1, 4):
        page = reader.pages[page_no]
        text = page.extract_text()
        print(text)
        return text

def parse_document(pdf_path):
    with open(pdf_path, 'rb') as file:

        reader = PdfReader(file)
        num_pages = len(reader.pages)

        index_text = table_of_content(reader)
        toc_data = toc_extractor.invoke({"paragraph": index_text})

        return toc_data
        

In [17]:
reader = PdfReader("../doc/load/2014-E1-En.pdf")
num_pages = len(reader.pages)
print(num_pages)
index_text = table_of_content(reader)
for page_no in range(1, 4):
    page = reader.pages[page_no]
    text = page.extract_text()
    print(text)


12






In [14]:
directory_path = '../doc/load'
table_of_content_list = list()

for filename in os.listdir(directory_path):
    print(f"Processing: {filename}")
    if filename.endswith('.pdf'): 
        file_path = os.path.join(directory_path, filename) 
        
        output = parse_document(file_path)
        table_of_content_list.append(output)
        print(f"Laws: {output.Laws}")
        print(f"Executive Council Resolutions: {output.ExecutiveCouncilResolutions}")
        print(f"Circulars: {output.Circulars}")
        print(f"Decrees: {output.Decrees}")

Processing: 2014-E1-En.pdf



BadRequestError: Error code: 400 - {'error': {'message': "Failed to generate JSON. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'json_validate_failed', 'failed_generation': '```json\n{\n  TFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFTFT

In [4]:
def table_of_content(reader):
    for page_no in range(1, 4):
        page = reader.pages[page_no]
        text = page.extract_text()
        return text

def parse_document(pdf_path):
    with open(pdf_path, 'rb') as file:

        reader = PdfReader(file)
        num_pages = len(reader.pages)

        index_text = table_of_content(reader)

index_text = parse_document('../doc/test')
index_text

PermissionError: [Errno 13] Permission denied: '../doc/test'

`2.2 Text Extraction and Doc Parsing`

In [44]:
from langchain.prompts import PromptTemplate
from PyPDF2 import PdfReader

from typing import List, Optional, Dict
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Index(BaseModel):
    Laws: Optional[List[str]] = Field(description="List of laws present in the legal document")
    Circulars: Optional[List[str]] = Field(description="List of circulars present in the legal document")
    Decrees: Optional[List[str]] = Field(description="List of decrees present in the legal document")
    ExecutiveCouncilResolutions: Optional[List[str]] = Field(description="List of executive council resolutions present in the legal document")


TOC_PROMPT = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an AI assistant specialized in identifying table of contents from the legal document paragraph.

    Guidelines:
    1. Thoroughly analyze the given document paragraph and generate the table of content.
    2. Table of content specifically target
        - Laws
        - Circulars
        - Decrees
        - ExecutiveCouncilResolutions
    3. Provide JSON response, where each dictionary key represents a primary topic (e.g., "Executive Council Resolutions," "Laws") and 
       each corresponding value contains a list of associated subtopics.
    4. Provide only the dictionary as the output, without any additional text, explanations, or commentary.
    5. If any primary topic is empty then exclude from the response and do not keep empty list.

    Sample response format:
    {{
      "Laws": ["Law No. (2) of 2022 Concerning the Reorganisation of the National Rehabilitation Centre - Abu Dhabi"],
      "ExecutiveCouncilResolutions": ["Chairman of the Executive Council Resolution No. (3) of 2022 Concerning the Appointment of the Director-General for Tourism"],
      "Circulars": ["Circular No. (1) of 2022 Concerning the Policy of Providing an Easy and Effortless Customer Experience in the Emirate of Abu Dhabi"],
      "Decrees": ["Amiri Decree No. (13) of 2023 Concerning Sending a Judge to Retirement"]
    }}

    Legal document paragraph: {paragraph}
    
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["paragraph"],
)

structured_llm = llm.with_structured_output(Index, method="json_mode")

toc_extractor = TOC_PROMPT | structured_llm

def table_of_content(reader):
    for page_no in range(1, 4):
        page = reader.pages[page_no]
        text = page.extract_text()
        return text

def parse_document(pdf_path):
    with open(pdf_path, 'rb') as file:

        reader = PdfReader(file)
        num_pages = len(reader.pages)

        index_text = table_of_content(reader)
        toc_data = toc_extractor.invoke({"paragraph": index_text})

        return toc_data
        

In [45]:
import os
import re
import json
from PyPDF2 import PdfReader

def remove_dot(text):
    return text[:-1] if text.endswith('.') else text

def extrating_content(key_topics, output_lower, final_output):

    for i in range(len(key_topics)):

        current_topic = key_topics[i]
        current_topic = remove_dot(current_topic)
        current_topic_lower = current_topic.lower()
        current_topic = current_topic_lower.replace("'", '"')

        if i < len(key_topics) - 1:
            next_topic = key_topics[i + 1]
            next_topic = remove_dot(next_topic)
            next_topic_lower = next_topic.lower()
            
            next_topic = next_topic_lower.replace("'", '"')

            start = output_lower.find(current_topic)
            end = output_lower.find(next_topic)

            substring = output_lower[start:end]
        else:
            start = output_lower.find(current_topic) + len(current_topic)
            end = len(output_lower)
            substring = output_lower[start:end]

        final_output[current_topic] = substring
    
    return final_output


def parse_document(pdf_path):

    # Reading the file from the pdf_path
    with open(pdf_path, 'rb') as file:

        overall_content = ""
        key_topics = list()
        topic_wise_content = {}

        # Initialize the PyPDF function
        reader = PdfReader(file)
        num_pages = len(reader.pages)

        # Extracting table of content
        index_text = table_of_content(reader)
        toc_data = toc_extractor.invoke({"paragraph": index_text})
        toc = json.loads(toc_data.json(exclude_none=True))
        for key, value in toc.items():
            key_topics.extend(value)
        
        # Extracting remain document content and preparing 
        for page_no in range(3, num_pages):
            page = reader.pages[page_no]
            text = page.extract_text()
            if text and text[0].isdigit():
                text = text[1:]
                text = re.sub(r'^\d+', '', text)

            cleaned_text = re.sub(r'[\n\r\t]', ' ', text)
            cleaned_text = cleaned_text.replace("  ", " ")
            cleaned_text = cleaned_text.replace("  ", " ")
            cleaned_text = cleaned_text.replace("”", "’")
            cleaned_text = cleaned_text.replace("“", "‘")

            overall_content = overall_content + " " + cleaned_text
            overall_content = overall_content.strip()
        
        # Extract the topic wise content from the document
        topic_wise_content = extrating_content(key_topics, overall_content.lower(), topic_wise_content)
        
        return toc, key_topics, overall_content, topic_wise_content
    
toc, key_topics, overall_content, topic_wise_content = parse_document("../doc/test/First Edition English 2023.pdf")

In [46]:
print(f"Table of content:\n{toc}\n\nIndex:\n{key_topics}\nOverall content:\n{overall_content}\n\nTopic wise Content:\n{topic_wise_content}")

Table of content:
{'Laws': ['Law No. (16) of 2022 Concerning The Licensing of The Drivers and Vehicles in The Emirate of Abu Dhabi.', 'Law No. (1) of 2023 Concerning the establishment of Abu Dhabi Transport Company ‘Public Joint Stock Company’.', 'Law No. (4) of 2023 Concerning The Establishment of Abu Dhabi Media Office.'], 'Circulars': [], 'Decrees': ['Amiri Decree No. (3) of 2023 Concerning Some Members of the Executive Council in the Emirate of Abu Dhabi.'], 'ExecutiveCouncilResolutions': ['Executive Council Resolution No. (169) of 2022 Concerning the Thirty-First Investment Zone in the Emirate of Abu Dhabi.', 'Executive Council Resolution No. (182) of 2022 Concerning the General Policy for the Management, Regulation and Protection of Groundwater in the Emirate of Abu Dhabi.', 'Executive Council Resolution No. (183) of 2022 Concerning the Amendment of Some Provisions of Chairman of the Executive Council Resolution No. (25) of 2006 Concerning the Issuance of the Executive Regulation

In [47]:
topic_wise_content

{'law no. (16) of 2022 concerning the licensing of the drivers and vehicles in the emirate of abu dhabi': 'law no. (16) of 2022 concerning the licensing of the drivers and vehicles in the emirate of abu dhabi we, mohamed bin zayed al nahyan, ruler of abu dhabi, having reviewed: • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments; • law no. (2) of 1971 concerning the national advisory council and its amendments; • law no. (19) of 2006 concerning the regulation of tansportation via taxis in the emirate of abu dhabi and its amendments; • law no. (30) of 2019 concerning the establishment of the department of municipalities and transport; • law no. (1) of 2020 concerning the general headquarters of abu dhabi police, and its amendments; • federal law no. (21) of 1995 concerning the traffic, and its amendments; and • based on what was presented to and approved by the executive council, issued the following law: article (1

`2.3 Initializing Embedding Model and function`

In [77]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sentence_transformers import SentenceTransformer

model_name = "nomic-ai/nomic-embed-text-v1"
model = SentenceTransformer(model_name, trust_remote_code=True)

def embedding_creation(input_texts, model):
    embedding = model.encode(input_texts)
    return embedding.tolist()

<All keys matched successfully>


In [78]:
for topic, content in topic_wise_content.items():
    embedding = embedding_creation(content, model)
    print(f"Topic: {topic}\n")
    print(f"Content: {content}\n")
    print(f"Embedding: {embedding}\n")

Topic: law no. (16) of 2022 concerning the licensing of the drivers and vehicles in the emirate of abu dhabi

Content: law no. (16) of 2022 concerning the licensing of the drivers and vehicles in the emirate of abu dhabi we, mohamed bin zayed al nahyan, ruler of abu dhabi, having reviewed: • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments; • law no. (2) of 1971 concerning the national advisory council and its amendments; • law no. (19) of 2006 concerning the regulation of tansportation via taxis in the emirate of abu dhabi and its amendments; • law no. (30) of 2019 concerning the establishment of the department of municipalities and transport; • law no. (1) of 2020 concerning the general headquarters of abu dhabi police, and its amendments; • federal law no. (21) of 1995 concerning the traffic, and its amendments; and • based on what was presented to and approved by the executive council, issued the following law

#### Reference
+ https://www.nomic.ai/blog/posts/nomic-embed-text-v1

`2.4 Data Enrichment`

In [102]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-8b-8192",
    # api_key="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",
    api_key="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"
)

In [126]:
class ReferenceSchema(BaseModel):
    Reference: Optional[List[str]] = Field(description="List of references like laws, Resolutions present in the legal document")
    Questions: Optional[List[str]] = Field(description="List of LLM generated questions")

REFERENCE_EXTRACTION_PROMPT = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id>
    You are an expert in analyzing legal document and identifying reference i.e., laws, resolutions from the provided document and Generating questions for Retrieval-Augmented Generation (RAG) systems

    Guidelines for reference identifying:
    1. Provide only references without additional information or summaries.
    2. Provide JSON response as a reference, corresponding value contains a list of associatedFormat the output as a Python list, e.g., ["resolution no. (106) of 2021 on economic licensing fees", "chairman of the executive council resolution no. (4) of 2021 concerning the reformation of the board of directors of abu dhabi housing authority", "law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments"].
    3. Make sure that you are including the resolution and law concerning in short summary
    4. Exclude self-reference {index} from the output.
    5. Present only law or reference numbers and their years.
    6. If a reference is not a law or resolution, ignore it and keep the empty Python list, e.g., []

    Guidelines for generating questions:
    1. Thoroughly analyze the entire document.
    2. Generate exactly 10 questions that cover various aspects and levels of complexity within the document's content.
    3. Create questions that specifically target:
      - Key facts and information
      - Main concepts and ideas
      - Relationships between different parts of the content
      - Potential applications or implications of the information
      - Comparisons or contrasts within the document
    4. Ensure questions require answers of varying lengths and complexity, from simple retrieval to more complex synthesis.
    5. Include questions that might require combining information from different parts of the document.
    6. Frame questions to test both literal comprehension and inferential understanding.
    7. Avoid yes/no questions; focus on open-ended questions that promote comprehensive answers.
    8. Consider including questions that might require additional context or knowledge to fully answer, to test the RAG system's ability to combine retrieved information with broader knowledge.
    9. Output only the ten questions, without any additional text, explanations, or answers.
    10.Provide JSON response as a reference, corresponding value contains a list of associatedFormat the output as a Python list

    Sample response format:
    {{
      "Reference": ["Law No. (2) of 2022", "Executive Council Resolution No. (3) of 2022", "Amiri Decree No. (13) of 2023", "Circular No. (1) of 2022", "law no. (1) of 1974", "law no. (2) of 2000"],
      "Questions": ["What is the primary purpose of Law No. (12) of 2021, as stated in the document?" , "What are the laws and regulations reviewed by the ruler of Abu Dhabi before issuing Law No. (12) of 2021?"]
    }}

    The document references {index} and contains the following content: {paragraph}
    
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["index", "paragraph"],
)

structured_llm = llm.with_structured_output(ReferenceSchema, method="json_mode")

reference_extraction = REFERENCE_EXTRACTION_PROMPT | structured_llm

In [127]:
import time
for topic, content in topic_wise_content.items():
    print(f"Topic: {topic}\n")
    print(f"Content: {content}\n")
    print(f"Content length: {len(content)}")
    reference = reference_extraction.invoke({"index":topic, "paragraph": content})
    print(f"Reference: {reference.Reference}\n")
    print(f"Questions: {reference.Questions}\n")
    # time.sleep(60)

Topic: law no. (13) of 2022 concerning the establishment of abu dhabi waste management company (tadweer) public joint stock company

Content: law no. (13) of 2022 concerning the establishment of abu dhabi waste management company (tadweer) public joint stock company we, mohamed bin zayed al nahyan, ruler of abu dhabi, having reviewed: • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments; • law no. (2) of 1971 concerning the national advisory council, and its amendments; • law no. (16) of 2005 concerning the reorganisation of the environment agency - abu dhabi, and its amendments; • law no. (21) of 2005 concerning waste management in the emirate of abu dhabi; • law no. (17) of 2008 concerning the establishment of abu dhabi waste management centre ; • law no. (2) of 2012 concerning the maintenance of the general appearance, health and public tranquillity of the emirate of abu dhabi, and its amendments; • law no. (1) o

In [128]:
directory_path = '../doc/'

final_index_document = []

for filename in os.listdir(directory_path):
    if filename.endswith('.pdf'): 

        file_path = os.path.join(directory_path, filename) 
        print(f"Processing File Name: {file_path}\n")

        toc, key_topics, overall_content, topic_wise_content = parse_document(file_path)
        print(f"Table of Content:\n{toc}\n")
        print(f"Key Topics:\n{key_topics}\n")
        print(f"Overall Content:\n{overall_content}\n")
        print(f"Topic-wise Content:\n{topic_wise_content}\n")

        for key, value in topic_wise_content.items():
            print(f"Topic:\n{key}\n")
            if len(value) > 50000:
                print(f"Skipped: {key}")
                continue

            embedding_text = embedding_creation(value, model)
            print(f"Embedding created\n")

            data_enrich = reference_extraction.invoke(
                    {
                        "index": key, 
                        "paragraph": value
                    }
            )
            references = data_enrich.Reference
            print(f"Reference:\n{references}\n")

            questions = reference.Questions
            print(f"Questions:\n{questions}\n")

            final_index_document.append(
                {
                    "heading": key,
                    "content": value,
                    "reference": references,
                    "questions": questions,
                    "embedded_content": embedding_text,
                }
            )

Processing File Name: ../doc/10English2021.pdf

Table of Content:
{'Laws': ['Law No. (12) of 2021 Concerning the Amendment of Some Provisions of Law No. (28) of 2007 Concerning the Reorganisation of The Zoo and Aquarium Public Institution in Al Ain.', 'Law No. (13) of 2021 Concerning the Amendment of Some Provisions of Law No. (5) of 1992 Concerning the Establishment of Zayed Bin Sultan Al Nahyan Charitable and Humanitarian Foundation.'], 'Circulars': [], 'Decrees': ['Amiri Decree No. (8) of 2020 Concerning Sending a Member of a Judicial Authority to Retirement'], 'ExecutiveCouncilResolutions': ['Chairman of the Executive Council Resolution No. (28) of 2021 Concerning The Reformation of the Board of Trustees of Zayed Bin Sultan Al Nahyan Charitable and Humanitarian Foundation.', 'Chairman of the Executive Council Resolution No. (29) of 2021 Concerning The Financial Support of Sports in the Emirate of Abu Dhabi.', 'Chairman of the Executive Council Resolution No. (30) of 2021 Concerning

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01jbaf7n3pfpz81079dy9nnfqc` on : Limit 500000, Used 497105, Requested 7333. Please try again in 12m46.7868s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': '', 'code': 'rate_limit_exceeded'}}

In [129]:
len(final_index_document)

151

`3. Creating new index in Elastic Search`

In [130]:
resp = helpers.bulk(
    es_client,
    final_index_document,
    chunk_size=20, 
    timeout="100m",
    index = "abudhabi-policies-legislations-v4",
)